## Infraestructuras Computacionales para el Procesamiento de Datos Masivos
### Práctica del Módulo 3: Gestión de datos en tiempo real (Streaming)
### Autor: Jesús Galán Llano
#### Correo: jgalan279@alumno.uned.es

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import explode

En primer lugar, creamos la sesión de Spark

In [2]:
spark = SparkSession \
    .builder \
    .appName("tp3") \
    .getOrCreate()

Tras esto, especificamos el tipo de entrada de datos que en este caso se corresponde con Apache Kafka. 
Es importante configurar correctamente el número de brokers de Kafka así como su dirección. La configuración descrita en la memoria
consiste de dos servidores, que están disponibles en los puertos 9092 y 9093 en el equipo local.

In [3]:
lines = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092,localhost:9093") \
  .option("subscribe", "kafkaTwitter") \
  .option("includeHeaders", "true") \
  .load()

Creamos un DataFrame formado por todas las palabras que aparecen en los tweets. Para ello, dividimos los mensajes en espacios en blanco
con la función split(). La función explode permite devolver más de un elemento, que es la lista de palabras, por cada elemento procesado, que es un tweet.

In [4]:
words = lines.select(
        explode(split(lines.value, ' ')).alias('word'),
    )

Como solamente nos interesa almacenar los hashtags aplicamos una operación de filtrado sobre el DataFrame anterior. 
Los hashtags son aquellas palabras que empiezan por el caracter '#'. 

In [5]:
hashtags = words.filter(words.word.startswith('#'))

Tras obtener todos los hashtags, se agrupan utilizando la función groupBy() y se realiza el cálculo de cuántas veces aparece dicho hashtag con la función count().
Seguidamente, ordenamos los resultados utilizando la función orderBy en base al número de veces que aparece cada hashtags de forma descendente, empezando por los que más se repiten.
Por último, limitamos los resultados a los 10 hashtags más utilizados con la función limit().

In [9]:
hashtagsCounts = hashtags.groupBy(
    hashtags.word
).count()\
    .orderBy('count', ascending=False)\
        .limit(10)

Por último, ejecutamos el proceso para procesar los tweets que van llegando desde el productor.
Se ha dejado la ejecución del programa durante un período de tiempo para validar su funcionamiento. Las conclusiones de la práctica se incluyen al final del cuaderno.

In [ ]:
hashtagsCounts\
        .writeStream\
        .outputMode('complete')\
        .format('console')\
        .option('truncate', 'false')\
        .start()

22/01/16 21:05:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-01f1d75b-e932-475e-b5ea-9e7819519a8e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/01/16 21:05:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+---------------+-----+
|word           |count|
+---------------+-----+
|#EscuelaPPA    |1    |
|#DebateTM      |1    |
|#AhoraPPCV     |1    |
|#representar   |1    |
|#liderar       |1    |
|#FelizMiercoles|1    |
|#digoSÍvotoPSOE|1    |
|#Nuñista       |1    |
|#TuitUtil      |1    |
|#servir        |1    |
+---------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------------------+-----+
|word               |count|
+-------------------+-----+
|#EscuelaPPA        |1    |
|#CD9enHitFM        |1    |
|#DebateTM          |1    |
|#AhoraPPCV         |1    |
|#EnPodemosTúDecides|1    |
|#L6Npartidos       |1    |
|#TrabajarHacerCreer|1    |
|#YoApoyoARafa      |1    |
|#representar       |1    |
|#liderar           |1    |
+-------------------+-----+



...

-------------------------------------------
Batch: 680
-------------------------------------------
+----------------------+-----+
|word                  |count|
+----------------------+-----+
|#UAM                  |128  |
|#ForoCambio11J        |120  |
|#UniDeVerano          |98   |
|#Apple                |93   |
|#TuitUtil             |85   |
|#TrabajarHacerCrecer  |85   |
|#Ciencia              |66   |
|#VotaPP               |62   |
|#destacada            |56   |
|#GobernarParaLaMayoría|46   |
+----------------------+-----+



-------------------------------------------
Batch: 681
-------------------------------------------
+----------------------+-----+
|word                  |count|
+----------------------+-----+
|#UAM                  |128  |
|#ForoCambio11J        |120  |
|#UniDeVerano          |98   |
|#Apple                |93   |
|#TuitUtil             |85   |
|#TrabajarHacerCrecer  |85   |
|#Ciencia              |67   |
|#VotaPP               |62   |
|#destacada            |56   |
|#GobernarParaLaMayoría|46   |
+----------------------+-----+



-------------------------------------------
Batch: 682
-------------------------------------------
+----------------------+-----+
|word                  |count|
+----------------------+-----+
|#UAM                  |128  |
|#ForoCambio11J        |120  |
|#UniDeVerano          |98   |
|#Apple                |94   |
|#TuitUtil             |85   |
|#TrabajarHacerCrecer  |85   |
|#Ciencia              |68   |
|#VotaPP               |62   |
|#destacada            |56   |
|#GobernarParaLaMayoría|46   |
+----------------------+-----+



-------------------------------------------
Batch: 683
-------------------------------------------
+----------------------+-----+
|word                  |count|
+----------------------+-----+
|#UAM                  |128  |
|#ForoCambio11J        |120  |
|#UniDeVerano          |98   |
|#Apple                |94   |
|#TuitUtil             |85   |
|#TrabajarHacerCrecer  |85   |
|#Ciencia              |68   |
|#VotaPP               |62   |
|#destacada            |57   |
|#GobernarParaLaMayoría|47   |
+----------------------+-----+



-------------------------------------------
Batch: 684
-------------------------------------------
+----------------------+-----+
|word                  |count|
+----------------------+-----+
|#UAM                  |128  |
|#ForoCambio11J        |120  |
|#UniDeVerano          |98   |
|#Apple                |94   |
|#TuitUtil             |85   |
|#TrabajarHacerCrecer  |85   |
|#Ciencia              |68   |
|#VotaPP               |62   |
|#destacada            |57   |
|#GobernarParaLaMayoría|47   |
+----------------------+-----+



## Conclusiones

Esta segunda parte de la práctica me ha resultado muy interesante porque me ha permitido poner en práctica mis conocimientos de Kafka y de SparkStreaming. Considero que
una de las características principales de un ingeniero de datos debe ser saber conectar más de un sistema para lograr un fin aprovechando las características de cada uno, como se realiza en esta práctica. 

A pesar de haberme atascado al comienzo luego he podido ir avanzando poco a poco empleando la extensa documentación que hay en internet sobre SparkStreaming
y consultando ciertos ejemplos de los libros de la bibliografía. Sin embargo, por falta de tiempo no he podido entregar el tercer ejercicio de esta parte por fala de tiempo.
Como punto a mejorar para los próximos años, comentar que los ejemplos incluidos en las transparencias respecto a Kafka están desactualizados porque funcionan con versiones de Spark inferiores a la versión 3 y requieren de Python2. Debido a esto he tenido que emplear más tiempo del deseado en preparar el entorno para la práctica y en encontrar una documentación que me fuera útil. Sería muy recomendable adaptar ese contenido al uso de Spark 3 y de Python 3 para aplicar todas las ventajas que estas versiones ofrecen. 

Por último, agradecer de nuevo las facilidades del equipo docente respecto a permitir entregas después de la fecha de entrega. 